# Hyperdrive Contract Integration

In [ ]:
from ape import accounts, chain, networks, Project
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

## Compilation and Network Setup

In [ ]:
networks.parse_network_choice('ethereum:local:foundry').__enter__()
project_root = Path.cwd().parent.parent
project = Project(path=project_root)

## Deploy Config

In [ ]:
# deployment config values

base_supply = int(1e23) # 100k
t_stretch = int(22.186877016851916266e18)
initial_rate = int(5e18) # share rate
initial_apr = int(.05e18)
share_price = int(1e18)
checkpoints = int(365)
checkpoint_duration = int(86400)
curve_fee = int(0)
flat_fee = int(0)

## Deploy Contracts

In [ ]:
# generate deployer account
deployer = accounts.test_accounts.generate_test_account()

# give deployer 3 eth
deployer.balance += int(3e18)

base_address = deployer.deploy(project.ERC20Mintable)
base_ERC20 = project.ERC20Mintable.at(base_address)


fixed_math_address = deployer.deploy(project.MockFixedPointMath)
fixed_math = project.MockFixedPointMath.at(fixed_math_address)

base_ERC20.mint(base_supply, sender=deployer)
time_stretch = fixed_math.divDown(int(1e18), t_stretch)

hyperdrive_address = deployer.deploy(project.MockHyperdriveTestnet, base_ERC20,
                                     initial_rate, share_price, checkpoints,
                                     checkpoint_duration, int(time_stretch), curve_fee,
                                     flat_fee)
hyperdrive = project.MockHyperdriveTestnet.at(hyperdrive_address)

with accounts.use_sender(deployer):
    base_ERC20.approve(hyperdrive, base_supply)
    hyperdrive.initialize(base_supply, initial_apr, deployer, False)


## User Config

In [ ]:
# User Config Values

user_balance = int(1e24)
long_amount = int(1e21) # 1000 base amount
min_out = int(1e21)
num_longs = 50

In [ ]:
# Generate hyperdrive user
hyperdrive_user = accounts.test_accounts.generate_test_account()

# Give user 1 eth balance
hyperdrive_user.balance += int(1e18)

# get current block
current_block = chain.blocks[-1].number

pool_info = [hyperdrive.getPoolInfo().__dict__]
pool_info[0]["block_number_"] = current_block

def open_longs():
    # with notation avoids passing in sender each time
    with accounts.use_sender(hyperdrive_user):
        # Mint 1 million DAI
        base_ERC20.mint(user_balance)

        # Approve ERC20 usage by contract
        base_ERC20.approve(hyperdrive, user_balance)

        pool_info = hyperdrive.getPoolInfo().__dict__

        # Open Long on 10,000 DAI
        tx_receipt = hyperdrive.openLong(long_amount, min_out, hyperdrive_user, False)
        pt_balance = tx_receipt.return_value
        
        print("user input %s base" %(long_amount))
        print("user received %s PTs" %(pt_balance))
        
        current_block = chain.blocks[-1]
        pool_info["block_number_"] = current_block.number

        return pool_info


for x in range(num_longs):        
    pool_info += [open_longs()]


In [ ]:
df = pd.DataFrame(pool_info)

plt.figure(1)
plt.plot(df['block_number_'], df['shareReserves_'], label='Share Reserves')
plt.plot(df['block_number_'], df['bondReserves_'], label='Bond Reserves')
plt.plot(df['block_number_'], df['lpTotalSupply'], label='LP Supply')
plt.xlabel('Block Number')
plt.ylabel('Reserves & Supply')
plt.title('Local Longs Opened Consecutively')
plt.legend()
plt.show()